## 실행 가능성 확인하기

In [4]:
import glob
import os
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

### 하이퍼파라미터

In [6]:
EPOCHS = 10

DATASET_PATH = 'data/dataset/2/'
DATASET_OK_PATTERN = DATASET_PATH + 'OK/*.png'
DATASET_FAIL_PATTERN = DATASET_PATH + 'FAIL/*.png'

RESULT_SAVE_PATE = 'data/results/'

### 단순 모델 설정

In [7]:
def Model():
    return Sequential([Conv2D(32, (3,3), activation='relu'), MaxPool2D(),
                       Conv2D(64, (3,3), activation='relu'), MaxPool2D(),
                       Conv2D(128, (3,3), activation='relu'), MaxPool2D(),
                       Conv2D(256, (3,3), activation='relu'), MaxPool2D(),
                       Flatten(),
                       Dense(1, activation='sigmoid')
                      ])

### 데이터셑 불러오기

In [10]:
def preprocess(file_name):
    img = tf.io.read_file(file_name)
    img = tf.image.decode_png(img)
    return tf.image.convert_image_dtype(img, tf.float32)

In [12]:
ok_list = glob.glob(DATASET_OK_PATTERN)
ds_ok = tf.data.Dataset.list_files(ok_list)
ds_ok_label = tf.data.Dataset.from_tensor_slices([0] * len(ok_list))

ds_ok = ds_ok.map(preprocess)
ds_ok = tf.data.Dataset.zip((ds_ok, ds_ok_label))

In [13]:
fail_list = glob.glob(DATASET_FAIL_PATTERN)
ds_fail = tf.data.Dataset.list_files(fail_list)
ds_fail_label = tf.data.Dataset.from_tensor_slices([1] * len(fail_list))

ds_fail = ds_fail.map(preprocess)
ds_fail = tf.data.Dataset.zip((ds_fail, ds_fail_label))

In [14]:
ds = tf.data.Dataset.concatenate(ds_ok, ds_fail)

### Train, Valid 데이터셑 나누기

In [15]:
ds_size = len(ok_list) + len(fail_list)
train_size = int(ds_size * 0.7)

ds = ds.shuffle(ds_size)
ds_train = ds.take(train_size).shuffle(1024, reshuffle_each_iteration=True).batch(32)
ds_valid = ds.skip(train_size).batch(32)

### 모델 생성 및 학습

In [16]:
model = Model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(ds_train, validation_data=ds_valid, epochs=EPOCHS)

Train for 22 steps, validate for 10 steps
Epoch 1/10
21/22 [===========================>..] - ETA: 1s - loss: 0.4360 - accuracy: 0.8705

### 결과를 이미지로 저장